In [1]:
import librosa
import librosa.display
import IPython.display as ipd
import matplotlib.pyplot as plt
import numpy as np
import os
import math
from sklearn.cluster import KMeans
import copy
import pickle
from sklearn.metrics import classification_report
import time
import os
from torch.utils.data import Dataset, DataLoader
import pandas as pd
import numpy as np
import random
from time import time
from collections import Counter 
from scipy.io import wavfile
from scipy import signal
import scipy
import torch
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
import torchaudio

In [2]:
!pip install -q wavencoder
import wavencoder

In [3]:
RATE = '8000' 

CATES = ['female', 'male']

BASE_ORIGINAL_TRAIN = '../input/gendersclassification-vdt-2022/dataset_v2/train'
NUM_WORKERS = 2
BASE_TRAIN = BASE_ORIGINAL_TRAIN
INFER_ONLY = True # change this to False to train the model again 
BASE_PUBLIC_TEST = '../input/gendersclassification-vdt-2022/public-test/public-test/wav'
BASE_PRIVATE_TEST = '../input/gendersclassification-vdt-2022/private-test/private-test/wav'

In [4]:
!ls

__notebook__.ipynb


In [5]:
audio_file='../input/gendersclassification-vdt-2022/dataset_v2/train/female/midside/-udGMvnQtZ0_100_8k.wav'

In [6]:
ipd.Audio(audio_file,autoplay=True)

In [7]:
# load audio files with librosa
signal, sr = librosa.load(audio_file)

In [8]:
print(sr)

22050


In [10]:
!ls $BASE_TRAIN

female	male


In [11]:
print(CATES)

['female', 'male']


In [15]:

from torch.utils.data import Dataset, DataLoader
import pandas as pd
from collections import Counter
from scipy import signal


def gen_spec(wav):

    linear = librosa.stft(wav, n_fft=2048, hop_length=512)
    features, _ = librosa.magphase(linear)
    features = librosa.power_to_db(features, ref=1.0, amin=1e-10, top_db=None)
    spectrogram = np.array(Image.fromarray(features).resize(size=(224, 224)))
    out = np.zeros((3, 224, 224), dtype = np.float32)
    out[0, :, :] = spectrogram
    out[1, :, :] = spectrogram 
    out[2, :, :] = spectrogram 
    return torch.Tensor(out)
    
class MyDatasetSTFT(Dataset):
    def __init__(self, filenames, labels, transform=None, duration=2, data_type = 'spectral'):
        assert len(filenames) == len(labels), "Number of files != number of labels"
        self.fns = filenames
        self.lbs = labels
        self.transform = transform
        self.duration = duration 
        self.noise_dataset_path='noise_dataset'
        self.get_transforms = wavencoder.transforms.Compose([
                wavencoder.transforms.PadCrop(pad_crop_length=duration*8000, pad_position='center', crop_position='random'),
        ])

    def __len__(self):
        return len(self.fns)

    def __getitem__(self, idx):
        fname = self.fns[idx]
        
        wav, samplerate = torchaudio.load(fname)
        transformed_audio = self.get_transforms(wav)
        transformed_audio = transformed_audio[0].cpu().numpy()
        
        feats = gen_spec(transformed_audio)
        return feats, self.lbs[idx], self.fns[idx]
# def get_fn_lbs():
#     lbs = []
#     fns = []
#     count = 0
#     for name_dir in os.listdir(BASE_TRAIN):
#         for i in os.listdir(os.path.join(BASE_TRAIN,name_dir)):
#             for j in os.listdir(os.path.join(BASE_TRAIN,name_dir,i)):
#                     dur = librosa.get_duration(filename=os.path.join(BASE_TRAIN,name_dir,i,j));
#                     if dur>1:
#                         lbs.append(count)
#                         fns.append(os.path.join(BASE_TRAIN,name_dir,i,j))
#             print(name_dir,i,end='')
#             print(count)
#             count+=1    
            
#     return lbs,fns




def get_fn_lbs():
    lbs = []
    fns = []
    for name_dir in os.listdir(BASE_TRAIN):
        for i in os.listdir(os.path.join(BASE_TRAIN,name_dir)):
            if name_dir == 'male':
                for j in os.listdir(os.path.join(BASE_TRAIN,name_dir,i)):
                    dur = librosa.get_duration(filename=os.path.join(BASE_TRAIN,name_dir,i,j))
                    if dur >1:
                        lbs.append(1)
                        fns.append(os.path.join(BASE_TRAIN,name_dir,i,j))
            if name_dir == 'female':
                for j in os.listdir(os.path.join(BASE_TRAIN,name_dir,i)):
                    dur = librosa.get_duration(filename=os.path.join(BASE_TRAIN,name_dir,i,j))
                    if dur >1:
                        lbs.append(0)
                        fns.append(os.path.join(BASE_TRAIN,name_dir,i,j))
    return lbs,fns

def get_fn_test():
    lbs = []
    fns = []
    df = pd.read_csv("../input/gendersclassification-vdt-2022/public-test/public-test/test_files.txt", header=None,names=["gender", "filename"])
    for index, row in df.iterrows():
        if str(row['gender']) == 'M':
            du=librosa.get_duration(filename=os.path.join(BASE_PUBLIC_TEST,str(row['filename'])))
            if du>1:
                lbs.append(1)
                fns.append(os.path.join(BASE_PUBLIC_TEST,str(row['filename'])))
        elif str(row['gender']) == 'F':
            du=librosa.get_duration(filename=os.path.join(BASE_PUBLIC_TEST,str(row['filename'])))
            if du>1:
                lbs.append(0)
                fns.append(os.path.join(BASE_PUBLIC_TEST,str(row['filename'])))
    return lbs,fns

def get_fn_submit():
    fns = []
    lbs = []
    df = pd.read_csv("../input/gendersclassification-vdt-2022/private-test/private-test/test_files.txt", header=None,names=["gender", "filename"])
    for index, row in df.iterrows():
        if str(row['gender']) == 'M':
            du=librosa.get_duration(filename=os.path.join(BASE_PRIVATE_TEST,str(row['filename'])))
            if du>1:
                lbs.append(1)
                fns.append(os.path.join(BASE_PRIVATE_TEST,str(row['filename'])))
        elif str(row['gender']) == 'F':
            du=librosa.get_duration(filename=os.path.join(BASE_PRIVATE_TEST,str(row['filename'])))
            if du>1:
                lbs.append(0)
                fns.append(os.path.join(BASE_PRIVATE_TEST,str(row['filename'])))
    return lbs,fns
    
def build_dataloaders(args):
    submit_lbs,submit_fns = get_fn_submit()
    
    lbs,fns = get_fn_lbs()
    
    train_fns, val_fns, train_lbs, val_lbs = train_test_split(fns, lbs, test_size=0.1, random_state=42,shuffle = True)
    test_lbs, test_fns = get_fn_test()
#     train_fns, test_fns, train_lbs, test_lbs =train_test_split(train_fns, train_lbs, test_size=0.1, random_state=42,shuffle = True)
    print('First val fn: {}'.format(val_fns[0]))
    print(Counter(train_lbs))
    print(Counter(val_lbs))

    num_classes = len(set(train_lbs))
    print('Total training files: {}'.format(len(train_fns)))
    print('Total validation files: {}'.format(len(val_fns)))
    print('Total classes: {}'.format(num_classes))

    dsets = dict()
    dsets['train'] =  MyDatasetSTFT(train_fns, train_lbs, duration = args.duration)
    dsets['val'] =  MyDatasetSTFT(val_fns, val_lbs, duration = args.duration)
    dsets['test'] = MyDatasetSTFT(test_fns, test_lbs, duration = args.duration)
    dsets['submit'] = MyDatasetSTFT(submit_fns, submit_lbs, duration = args.duration)
    
    dset_loaders = dict() 
    dset_loaders['train'] = DataLoader(dsets['train'],
            batch_size = args.batch_size,
            shuffle = True,
            num_workers = NUM_WORKERS)

    dset_loaders['val'] = DataLoader(dsets['val'],
            batch_size = args.batch_size,
            shuffle = False,
            num_workers = NUM_WORKERS)

    dset_loaders['test'] = DataLoader(dsets['test'],
            batch_size = args.batch_size,
            shuffle = False,
            num_workers = NUM_WORKERS)
    
    dset_loaders['submit'] = DataLoader(dsets['submit'],
            batch_size = args.batch_size,
            shuffle = False,
            num_workers = NUM_WORKERS)
    
    return dset_loaders, (train_fns, test_fns, val_fns, train_lbs, test_lbs, val_lbs)


In [16]:
import torchvision.models as models

import torch.optim as optim
from PIL import Image
import numpy as np



def net_frozen(args, model):
    print('********************************************************')
    model.frozen_until(args.frozen_until)
    init_lr = args.lr
    if args.trainer.lower() == 'adam':
        optimizer = optim.Adam(filter(lambda p: p.requires_grad, model.parameters()), 
                lr=init_lr, weight_decay=args.weight_decay)
    elif args.trainer.lower() == 'sgd':
        optimizer = optim.SGD(filter(lambda p: p.requires_grad, model.parameters()), 
                lr=init_lr,  weight_decay=args.weight_decay)
    print('********************************************************')
    return model, optimizer

def parallelize_model(model):
    if torch.cuda.is_available():
        model = model.cuda()
        model = torch.nn.DataParallel(model, device_ids=range(torch.cuda.device_count()))
        cudnn.benchmark = True
    return model

def unparallelize_model(model):
    try:
        while 1:
            # to avoid nested dataparallel problem
            model = model.module
    except AttributeError:
        pass
    return model

def second2str(second):
    h = int(second/3600.)
    second -= h*3600.
    m = int(second/60.)
    s = int(second - m*60)
    return "{:d}:{:02d}:{:02d} (s)".format(h, m, s)

def print_eta(t0, cur_iter, total_iter):
    """
    print estimated remaining time
    t0: beginning time
    cur_iter: current iteration
    total_iter: total iterations
    """
    time_so_far = time() - t0
    iter_done = cur_iter + 1
    iter_left = total_iter - cur_iter - 1
    second_left = time_so_far/float(iter_done) * iter_left
    s0 = 'Epoch: '+ str(cur_iter + 1) + '/' + str(total_iter) + ', time so far: ' \
        + second2str(time_so_far) + ', estimated time left: ' + second2str(second_left)
    print(s0)

def get_length_wav(fn):
    frame_rate, signal = wavfile.read(fn)
    return float(signal.shape[0])/frame_rate



In [17]:
import torch.nn as nn

class MyResNet(nn.Module):
    def __init__(self, depth, num_classes, pretrained = True):
        super(MyResNet, self).__init__()
        if depth == 18:
            model = models.resnet18(pretrained)
        elif depth == 34:
            model = models.resnet34(pretrained)
        elif depth == 50:
            model = models.resnet50(pretrained)
        elif depth == 152:
            model = models.resnet152(pretrained)

        self.num_ftrs = model.fc.in_features

        self.shared = nn.Sequential(*list(model.children())[:-1])
        self.target = nn.Sequential(nn.Linear(self.num_ftrs, num_classes))

    def forward(self, x):


        x = self.shared(x)
        x = torch.squeeze(x)
        return self.target(x)

    def frozen_until(self, to_layer):
        print('Frozen shared part until %d-th layer, inclusive'%to_layer)
        child_counter = 0
        for child in self.shared.children():
            if child_counter <= to_layer:
                print("child ", child_counter, " was frozen")
                for param in child.parameters():
                    param.requires_grad = False
                # frozen deeper children? check
                # https://spandan-madan.github.io/A-Collection-of-important-tasks-in-pytorch/
            else:
                print("child ", child_counter, " was not frozen")
                for param in child.parameters():
                    param.requires_grad = True
            child_counter += 1



In [18]:
from __future__ import division
from sklearn.model_selection import train_test_split

import argparse

"""
Predict output of a new sample given model/models and fn/fns/dset
"""

def softmax_stable(Z):
    """
    Compute softmax values for each sets of scores in Z.
    each row of Z is a set of score.    
    """
    e_Z = np.exp(Z - np.max(Z, axis = 1, keepdims = True)) 
    A = e_Z / e_Z.sum(axis = 1, keepdims = True)
    return A

def loader_len(dset):
    """
    return len of a DataLoader 

    dset: a DataLoader that return (data, lbs, fns) at a batch
    """
    res = 0 
    for _, _, fns in dset:
        res += len(fns)
    return res 

def get_num_classes(model):
    """
    return number of output class of a pytorch classification model
    """
    # Step 1: unparalell_model
    model = unparallelize_model(model)
    return model.target[0].out_features


def singlemodel_score(model, dset_loader, num_tests = 1):
    """
    Use ONE pretrained model to predict score and probs each input in dset 
    Make multiples predictions and accumulate results

    ----
    INPUT: 
        model: a pretrained model
        dset: a MyDatasetSTFT variable
    OUTPUT:
        pred_outputs: np array -- prediction results, sum of all ouput before softmax  
        pred_probs: np array -- prediction results, sum of all probability  
        fns: list of filenames in loader order
    """
    num_files = loader_len(dset_loader)

    num_classes = 2


    total_scores = np.zeros((num_files, num_classes)) 
    total_probs = np.zeros((num_files, num_classes))
    fns = []
    torch.set_grad_enabled(False)
    model.eval()

    for test in range(num_tests):
        tot = 0 
        print('test {}/{}'.format(test + 1, num_tests))
        start = 0
        for batch_idx, (inputs, labels, fns0) in enumerate(dset_loader):
            n = len(fns0)
            inputs = inputs.to(device)
            output = model(inputs)
            output = output.view((-1, num_classes))
            _, pred  = torch.max(output.data, 1)
            total_scores[start:start + n, :] += output.data.cpu().numpy()
            start += n
            tot += len(fns0)
            if test == 0: fns.extend(fns0)

        total_probs += softmax_stable(total_scores)
    return total_scores, total_probs, fns


def singlemodel_class(model, dset_loader, num_tests = 1):
    """
    predict label for dset_loader using one model
    This one is done after calling sm_score(model, dset_loader, num_tests)
    and get the torch.max(out.data, 1)
    """
    total_scores, total_probs, fns = singlemodel_score(model, dset_loader, num_tests)
    score_class = np.argmax(total_scores, axis = 1) 
    prob_class = np.argmax(total_probs, axis = 1)
    return score_class, prob_class, fns

In [19]:
parser = argparse.ArgumentParser(description='PyTorch Digital Mammography Training')

parser.add_argument('--lr', default=1e-3, type=float, help='learning rate')
parser.add_argument('--net_type', default='resnet', type=str, help='model')
parser.add_argument('--depth', default=50, choices = [11, 16, 19, 18, 34, 50, 152, 161, 169, 121, 201], type=int, help='depth of model')
parser.add_argument('--weight_decay', default=5e-4, type=float, help='weight decay')
parser.add_argument('--finetune', '-f', action='store_true', help='Fine tune pretrained model')
parser.add_argument('--trainer', default='adam', type = str, help = 'optimizer')
parser.add_argument('--duration', default= 2, type = float, help='time duration for each file in second')
parser.add_argument('--n_tests', default=3, type = int, help='number of tests in valid set')
parser.add_argument('--gender', '-g', action='store_true', help='classify gender')
parser.add_argument('--accent', '-a', action='store_true', help='accent classifier')
parser.add_argument('--random_state', '-r', default = 2, type = int, help='random state in train_test_split')

parser.add_argument('--model_path', type=str, default = ' ')
parser.add_argument('--gamma', default = 0.5, type = float)
parser.add_argument('--batch_size', default=128, type=int)
parser.add_argument('--num_epochs', default=30, type=int,
                    help='Number of epochs in training')
parser.add_argument('--dropout_keep_prob', default=0.5, type=float)
parser.add_argument('--check_after', default=5,
                    type=int, help='check the network after check_after epoch')
parser.add_argument('--train_from', default=1,
                    choices=[0, 1],  # 0: from scratch, 1: from pretrained 1 (need model_path)
                    type=int,
                    help="training from beginning (1) or from the most recent ckpt (0)")

parser.add_argument('--frozen_until', '-fu', type=int, default = -1,
                    help="freeze until --frozen_util block")
parser.add_argument('--val_ratio', default=0.1, type=float, 
        help = "number of training samples per class")

########################################################################################33

args, unknown = parser.parse_known_args()

In [20]:
from sklearn.metrics import accuracy_score
import torch
import torch.nn as nn
import copy
import torch.backends.cudnn as cudnn
import os, sys
from time import time, strftime
from scipy.io import wavfile
from sklearn.metrics import confusion_matrix
import warnings
warnings.filterwarnings("ignore", message="numpy.dtype size changed")
warnings.filterwarnings("ignore", message="numpy.ufunc size changed")


print('======================================================')
print('Data preparation')
dset_loaders, train_info = build_dataloaders(args)
train_fns, semi_fns, val_fns, train_lbs, semi_lbs, val_lbs = train_info
num_classes = len(set(train_lbs))

def exp_lr_scheduler(args, optimizer, epoch):
    # after epoch 100, not more learning rate decay
    init_lr = args.lr
    lr_decay_epoch = 4 # decay lr after each 10 epoch
    weight_decay = args.weight_decay
    lr = init_lr * (0.6 ** (min(epoch, 200) // lr_decay_epoch)) 

    for param_group in optimizer.param_groups:
        param_group['lr'] = lr
        param_group['weight_decay'] = weight_decay

    return optimizer, lr

saved_models = './saved_model/'
if not os.path.isdir(saved_models): os.mkdir(saved_models)
saved_model_fn = saved_models + args.net_type + '_' +\
    str(args.depth) + '_' +  strftime('%m%d_%H%M') + '_r' + str(args.random_state) 
print('model will be saved to {}'.format(saved_model_fn))
print('********************************************************')
old_model = './checkpoint/' + args.net_type + '_' + str(args.depth) + '_' +   args.model_path + '.t7'
if args.train_from == 1 and os.path.isfile(old_model):
    print("| Load pretrained at  %s..." % old_model)
    checkpoint = torch.load(old_model, map_location=lambda storage, loc: storage)
    tmp = checkpoint['model']
    model = unparallelize_model(tmp)
    try:
        top1acc = checkpoint['acc']
        print('previous acc\t%.4f'% top1acc)
    except KeyError:
        pass
    print('=============================================')
else:
    model = MyResNet(args.depth, num_classes)



Data preparation
First val fn: ../input/gendersclassification-vdt-2022/dataset_v2/train/male/southside/os4a8CZPgCA_181_8k.wav
Counter({1: 6871, 0: 6201})
Counter({1: 769, 0: 684})
Total training files: 13072
Total validation files: 1453
Total classes: 2
model will be saved to ./saved_model/resnet_50_0621_0552_r2
********************************************************


Downloading: "https://download.pytorch.org/models/resnet50-0676ba61.pth" to /root/.cache/torch/hub/checkpoints/resnet50-0676ba61.pth


  0%|          | 0.00/97.8M [00:00<?, ?B/s]

In [21]:
# pip install torch-summary

In [22]:
# from torchsummary import summary
# summary(model, (3,224,224), device=device)a

In [23]:
model, optimizer = net_frozen(args, model)
model = parallelize_model(model)
criterion = nn.CrossEntropyLoss()
################################ 
N_train = len(train_lbs)
N_valid = len(val_lbs)
best_acc = 0
########## Start training
print('Start training ... ')
t0 = time()
for epoch in range(args.num_epochs):
    optimizer, lr = exp_lr_scheduler(args, optimizer, epoch) 
    print('#################################################################')
    print('=> Training Epoch #%d, LR=%.10f' % (epoch + 1, lr))
    running_loss, running_corrects, tot = 0.0, 0.0, 0.0
    running_loss_src, running_corrects_src, tot_src = 0.0, 0.0, 0.0
    ########################
    model.train()
    torch.set_grad_enabled(True)
    ## Training 
    for batch_idx, (inputs, labels, _) in enumerate(dset_loaders['train']):
        optimizer.zero_grad()
        inputs = inputs.to(device)
        labels = labels.to(device)
        outputs = model(inputs)
        
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        ############################################
        a, preds = torch.max(outputs.data, 1)
        running_loss += loss.item()
        running_corrects += preds.eq(labels.data).cpu().sum()
        tot += labels.size(0)
        sys.stdout.write('\r')
        try:
            batch_loss = loss.item()
        except NameError:
            batch_loss = 0

        top1acc = float(running_corrects)/tot
        sys.stdout.write('| Epoch [%2d/%2d] Iter [%3d/%3d]\tBatch loss %.4f\tTop1acc %.4f'
                         % (epoch + 1, args.num_epochs, batch_idx + 1,
                            (len(train_fns) // args.batch_size), batch_loss/args.batch_size,
                            top1acc))
        sys.stdout.flush()
        sys.stdout.write('\r')

    top1acc =  float(running_corrects)/N_train
    epoch_loss = running_loss/N_train
    print('\n| Training loss %.8f\tTop1error %.4f'\
            % (epoch_loss, top1acc))

    print_eta(t0, epoch, args.num_epochs)

    ###################################
    ## Validation
    if (epoch + 1) % args.check_after == 0:
        # Validation 
        ###################### 
        n_files = len(val_lbs)
        print('On test set')
        pred_output, pred_prob, _ = singlemodel_class(model, dset_loaders['test'], num_tests = 2)
        print(confusion_matrix(semi_lbs, pred_output))
        acc1 = accuracy_score(semi_lbs, pred_output)
        acc2 = accuracy_score(semi_lbs, pred_prob)
        print('acc_output: {}, acc_prob: {}'.format(acc1, acc2))
        print('On validation')
        pred_output, pred_prob, _ = singlemodel_class(model, dset_loaders['val'], num_tests =args.n_tests)
        print(confusion_matrix(val_lbs, pred_output))
        acc1 = accuracy_score(val_lbs, pred_output)
        acc2 = accuracy_score(val_lbs, pred_prob)
        print('acc_output: {}, acc_prob: {}'.format(acc1, acc2))
        ########## end test on multiple windows ##############3
        running_loss, running_corrects, tot = 0.0, 0.0, 0.0
        torch.set_grad_enabled(False)
        model.eval()
        for batch_idx, (inputs, labels, _) in enumerate(dset_loaders['val']):
            inputs = inputs.to(device)
            labels = labels.to(device)
            outputs = model(inputs)
            
            _, preds  = torch.max(outputs.data, 1)
            running_loss += loss.item()
            running_corrects += preds.eq(labels.data).cpu().sum()
            tot += labels.size(0)

        epoch_loss = running_loss / N_valid 
        top1acc= float(running_corrects)/N_valid
        # top3error = 1 - float(runnning_topk_corrects)/N_valid
        print('| Validation loss %.8f\tTop1acc %.4f'\
                % (epoch_loss, top1acc))

        ################## save model based on best acc 
        if acc1 > best_acc:
            best_acc = acc1
            print('Saving model')
            best_model = copy.deepcopy(model)
            state = {
                'model': best_model,
                'acc' : acc1,
                'clipped': args.duration,
                'args': args
            }

            torch.save(state, saved_model_fn + '.t7')
            print('=======================================================================')
            print('model saved to %s' % (saved_model_fn + '.t7'))



********************************************************
Frozen shared part until -1-th layer, inclusive
child  0  was not frozen
child  1  was not frozen
child  2  was not frozen
child  3  was not frozen
child  4  was not frozen
child  5  was not frozen
child  6  was not frozen
child  7  was not frozen
child  8  was not frozen
********************************************************
Start training ... 
#################################################################
=> Training Epoch #1, LR=0.0010000000
| Epoch [ 1/30] Iter [103/102]	Batch loss 0.0002	Top1acc 0.9379
| Training loss 0.00141178	Top1error 0.9379
Epoch: 1/30, time so far: 0:02:09 (s), estimated time left: 1:02:27 (s)
#################################################################
=> Training Epoch #2, LR=0.0010000000
| Epoch [ 2/30] Iter [103/102]	Batch loss 0.0008	Top1acc 0.9655
| Training loss 0.00091225	Top1error 0.9655
Epoch: 2/30, time so far: 0:04:02 (s), estimated time left: 0:56:36 (s)
#########################

In [24]:
running_loss, running_corrects, tot = 0.0, 0.0, 0.0
lbs = []
pred = []
model.eval()
for batch_idx, (inputs, labels, _) in enumerate(dset_loaders['test']):
    lbs+=labels
    
    inputs = inputs.to(device)
    labels = labels.to(device)
    outputs = best_model(inputs)
    _, preds  = torch.max(outputs.data, 1)
    running_loss += loss.item()
    running_corrects += preds.eq(labels.data).cpu().sum()
    tot += labels.size(0)
    pred+=preds.cpu()
top1acc= float(running_corrects)/N_valid
print('| Validation loss %.8f\tTop1acc %.4f'\
            % (epoch_loss, top1acc))

| Validation loss 0.00001214	Top1acc 1.9140


In [25]:
print(classification_report(lbs,pred))

              precision    recall  f1-score   support

           0       0.97      0.98      0.97      1500
           1       0.97      0.96      0.97      1368

    accuracy                           0.97      2868
   macro avg       0.97      0.97      0.97      2868
weighted avg       0.97      0.97      0.97      2868

